In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.model_zoo as model_zoo
import math
from pytorch.mil import MIL
from pytorch.data import get_training_set, get_test_set

In [2]:
vgg16 = models.vgg16()
vgg16_features = next(vgg16.children())
net = MIL(vgg16_features)

In [6]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

trainset = get_training_set()
testset = get_test_set()
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=1)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=1)

In [8]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    print(img)
    #img = img / 2.0 + 0.5
    #npimg = img.numpy()
    #plt.imshow(np.transpose(npimg, (1, 2, 0)))
    #plt.show()
    
dataiter = iter(trainloader)
images, labels = dataiter.next()

for img in images:
    middle_slice = img[30]
    imshow(middle_slice)

print(' '.join('{label}'.format(label=labels[j]) for j in range(4)))


-10000 -10000 -10000  ...  -10000 -10000 -10000
-10000 -10000 -10000  ...  -10000 -10000 -10000
-10000 -10000 -10000  ...  -10000 -10000 -10000
        ...            ⋱           ...         
-10000 -10000 -10000  ...  -10000 -10000 -10000
-10000 -10000 -10000  ...  -10000 -10000 -10000
-10000 -10000 -10000  ...  -10000 -10000 -10000
[torch.ShortTensor of size 227x227]


-10000 -10000 -10000  ...  -10000 -10000 -10000
-10000 -10000 -10000  ...  -10000 -10000 -10000
-10000 -10000 -10000  ...  -10000 -10000 -10000
        ...            ⋱           ...         
-10000 -10000 -10000  ...  -10000 -10000 -10000
-10000 -10000 -10000  ...  -10000 -10000 -10000
-10000 -10000 -10000  ...  -10000 -10000 -10000
[torch.ShortTensor of size 227x227]


-10000 -10000 -10000  ...  -10000 -10000 -10000
-10000 -10000 -10000  ...  -10000 -10000 -10000
-10000 -10000 -10000  ...  -10000 -10000 -10000
        ...            ⋱           ...         
-10000 -10000 -10000  ...  -10000 -10000 -10000
-10000 -100

In [15]:
cross_entropy = nn.CrossEntropyLoss()
l1 = nn.L1Loss(size_average=False)
def criterion(output, labels, w, sparsity):
    pred, mil = output
    loss = w * cross_entropy(pred)
    loss += sparsity * l1(mil)
    
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=.9)

In [18]:
net = net.cuda()
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        labels = (labels > 4) * 1
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
        if i % 2000 == 1999:
            print('[{}, {}] loss: {}'.format(epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('finished training')

RuntimeError: Given input size: (1x1x1). Calculated output size: (1x0x0). Output size is too small at /b/wheel/pytorch-src/torch/lib/THCUNN/generic/SpatialDilatedMaxPooling.cu:69